In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from pathlib import Path
dir_path = Path().cwd()
inp_path = dir_path /'input'
for dirname, _, filenames in os.walk(inp_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

D:\work\ffwork\nlp\nlp1\input\sample_submission.csv
D:\work\ffwork\nlp\nlp1\input\test.csv
D:\work\ffwork\nlp\nlp1\input\train.csv
D:\work\ffwork\nlp\nlp1\input\twitter_training.csv
D:\work\ffwork\nlp\nlp1\input\twitter_training.csv.zip
D:\work\ffwork\nlp\nlp1\input\twitter_validation.csv
D:\work\ffwork\nlp\nlp1\input\vocab.txt


In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

In [27]:
# train_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
# test_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
# train_df.head()
train_df = pd.read_csv('./input/twitter_training.csv', header=None)
val_df = pd.read_csv('./input/twitter_validation.csv', header=None)

In [28]:
train_df.reset_index(drop=True,inplace=True)
val_df.reset_index(drop=True,inplace=True)
df = pd.concat([train_df,val_df], axis=0)

In [29]:
df.drop([0], axis=1, inplace=True)

In [30]:
df.columns = ['platform','sentiment','text']

In [31]:
df

,platform,sentiment,text
0,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,Borderlands,Positive,I am coming to the borders and I will kill you...
2,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,Borderlands,Positive,im coming on borderlands and i will murder you...
4,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...
995,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
996,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
997,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
998,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [32]:
df.drop(['platform'], axis=1, inplace=True)
df.sentiment = df.sentiment.map({"Neutral":0, "Irrelevant":0 ,"Positive":1,"Negative":2})

In [33]:
df.isna().sum()

sentiment      0
text         686
dtype: int64

In [34]:
df.dropna(inplace=True)

Data Cleaning

In [35]:
#import nltk
#nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\南京中略信息技术有限公司\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Data Cleaning

In [36]:
stop_words = set(stopwords.words('english'))
def data_preprocessing(text):
    text = text.lower()
    text = re.sub('<.*?>', '', text) # Remove HTML from text
    text = ''.join([c for c in text if c not in string.punctuation])# Remove punctuation
    text = [word for word in text.split() if word not in stop_words]
    text = ' '.join(text)
    return text

df['text'] = df['text'].astype(str).apply(data_preprocessing)
df.head()

,sentiment,text
0,1,im getting borderlands murder
1,1,coming borders kill
2,1,im getting borderlands kill
3,1,im coming borderlands murder
4,1,im getting borderlands 2 murder


Tokenization

In [37]:
corpus = [word for text in df['text'] for word in text.split()]
count_words = Counter(corpus)
sorted_words = count_words.most_common()

In [38]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

text_int = []
for text in df['text']:
    r = [vocab_to_int[word] for word in text.split()]
    text_int.append(r)

df['text int'] = text_int

In [39]:
text_len = [len(x) for x in text_int]
max_len = max(text_len)
print(max_len)

163


Padding

Padding

In [40]:
def Padding(text_int, seq_len):
    '''
    Return features of text_ints, where each text is padded with 0's
    '''
    features = np.zeros((len(text_int), seq_len), dtype = int)
    for i, text in enumerate(text_int):
        if len(text) <= seq_len:
            zeros = list(np.zeros(seq_len - len(text)))
            new = zeros + text
        features[i, :] = np.array(new)
            
    return features

In [41]:
features = Padding(text_int, max_len)

print(features[0, :])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    5   83   57 1610]


Data preparation

Data preparation

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_remain, y_train, y_remain = train_test_split(features, df['sentiment'].to_numpy(), test_size=0.2, random_state=1)
X_valid, X_test, y_valid, y_test = train_test_split(X_remain, y_remain, test_size=0.5, random_state=1)

In [43]:
# create tensor dataset
train_data = TensorDataset(torch.from_numpy(X_train.astype(float)), torch.from_numpy(y_train.astype(float)))
test_data = TensorDataset(torch.from_numpy(X_test.astype(float)), torch.from_numpy(y_test.astype(float)))
valid_data = TensorDataset(torch.from_numpy(X_valid.astype(float)), torch.from_numpy(y_valid.astype(float)))

# dataloaders
batch_size = 32

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [44]:
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample output size: ', sample_y.size())
print('Sample input: \n', sample_x)
print('Sample input: \n', sample_y)

Sample input size:  torch.Size([32, 163])
Sample output size:  torch.Size([32])
Sample input: 
 tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.1960e+03, 5.3200e+02,
         9.9440e+03],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 2.8210e+03, 1.9980e+03,
         3.7773e+04],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 5.9000e+01, 1.2000e+03,
         7.3700e+02],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 1.0800e+02,
         5.8000e+02],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.7590e+03, 1.3900e+02,
         3.6220e+04],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 5.3100e+02, 2.6000e+01,
         2.1400e+03]], dtype=torch.float64)
Sample input: 
 tensor([2., 0., 0., 1., 1., 2., 0., 0., 2., 0., 0., 0., 0., 0., 2., 2., 1., 0.,
        2., 0., 0., 1., 2., 0., 2., 1., 2., 0., 0., 0., 0., 2.],
       dtype=torch.float64)


Define model class

In [45]:
class sentimentLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers):
        """
        Initialize the model by setting up the layers.
        """
        super(sentimentLSTM, self).__init__()
        
        self.output_size = output_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        # Embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)
        

        self.fc = nn.Linear(hidden_dim, output_size)
        

        
    def forward(self, x, hidden):

        embeds = self.embedding(x.int())
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out[:, -1]
        out = self.fc(lstm_out)

        return out
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        h0 = torch.zeros((self.n_layers,batch_size,self.hidden_dim)).to(device)
        c0 = torch.zeros((self.n_layers,batch_size,self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden


In [46]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int) + 1
output_size = 3
embedding_dim = 64
hidden_dim = 256
n_layers = 1

model = sentimentLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model = model.to(device)

print(model)

GPU is available
sentimentLSTM(
  (embedding): Embedding(42037, 64)
  (lstm): LSTM(64, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=3, bias=True)
)


Model training

Model training

In [47]:
from torch import optim

optimizer = optim.Adam(model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss() # multi-class, if binary, then use BCELoss()

def acc(pred,label):
    pred = pred.argmax(1)
    return torch.sum(pred == label.squeeze()).item()

In [48]:
clip = 5
epochs = 5 
valid_loss_min = np.Inf
# train for some number of epochs
epoch_tr_loss,epoch_vl_loss = [],[]
epoch_tr_acc,epoch_vl_acc = [],[]

for epoch in range(epochs):
    train_losses = []
    train_acc = 0.0
    model.train()
    correct = 0
    # initialize hidden state 
    h = model.init_hidden(batch_size)
    for inputs, labels in train_loader:
        
        inputs, labels = inputs.to(device), labels.to(device)   
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])
        model.zero_grad()
        output = model(inputs,h)
        # calculate the loss and perform backprop
        loss = criterion(output, labels.long())
        loss.backward()
        train_losses.append(loss.item())
        # accuracy
        accuracy = acc(output,labels)
        train_acc += accuracy
        #`clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        
    val_h = model.init_hidden(batch_size)
    val_losses = []
    val_acc = 0.0
    model.eval()
    for inputs, labels in valid_loader:
            val_h = tuple([each.data for each in val_h])

            inputs, labels = inputs.to(device), labels.to(device)

            output = model(inputs, val_h)
            val_loss = criterion(output, labels.long())

            val_losses.append(val_loss.item())
            
            accuracy = acc(output,labels)
            val_acc += accuracy
            
    epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(val_losses)
    epoch_train_acc = train_acc/len(train_loader.dataset)
    epoch_val_acc = val_acc/len(valid_loader.dataset)
    epoch_tr_loss.append(epoch_train_loss)
    epoch_vl_loss.append(epoch_val_loss)
    epoch_tr_acc.append(epoch_train_acc)
    epoch_vl_acc.append(epoch_val_acc)
    print(f'Epoch {epoch+1}') 
    print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')
    print(f'train_accuracy : {epoch_train_acc*100} val_accuracy : {epoch_val_acc*100}')
    if epoch_val_loss <= valid_loss_min:
        torch.save(model.state_dict(), './working/state_dict.pt')
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
        valid_loss_min = epoch_val_loss
    print(25*'==')

Epoch 1
train_loss : 0.8693432961704891 val_loss : 0.7009588476174917
train_accuracy : 58.97726515101007 val_accuracy : 69.49333333333333
Validation loss decreased (inf --> 0.700959).  Saving model ...
Epoch 2
train_loss : 0.49242197117880543 val_loss : 0.4571829812012167
train_accuracy : 80.2370158010534 val_accuracy : 82.33333333333334
Validation loss decreased (0.700959 --> 0.457183).  Saving model ...
Epoch 3
train_loss : 0.21958540033115712 val_loss : 0.4024176432026757
train_accuracy : 91.78111874124941 val_accuracy : 85.78666666666666
Validation loss decreased (0.457183 --> 0.402418).  Saving model ...
Epoch 4
train_loss : 0.110486141282751 val_loss : 0.4201985192286153
train_accuracy : 95.73638242549504 val_accuracy : 86.06666666666666
Epoch 5
train_loss : 0.08215279881323101 val_loss : 0.45022211465825385
train_accuracy : 96.70144676311753 val_accuracy : 86.61333333333333


Testing Loop

In [49]:
# Get test data loss and accuracy

test_losses = [] # track loss
# num_correct = 0

test_acc = 0.0

# init hidden state
test_h = model.init_hidden(batch_size)

model.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    test_h = tuple([each.data for each in test_h])

    inputs, labels = inputs.to(device), labels.to(device)
    
    output = model(inputs, test_h)
    
    # calculate loss
    test_loss = criterion(output, labels.long())
    test_losses.append(test_loss.item())
    
    accuracy = acc(output,labels)
    test_acc += accuracy


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = test_acc/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc*100))

# epoch_test_acc = test_acc/len(test_loader.dataset)

Test loss: 0.411
Test accuracy: 87.293
